### 1. Setup: Libraries & Configuration

In [1]:
#Standard libraries
import sys
sys.path.append('../src')

#Third party libraries
from itertools import permutations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

#Custom libraries
from config import YFINANCE_CLEAN
from constants import contract_sizes, tons_conversion
from strategy import backtest
from utils import pnl_trades, backtest_performance, strategy_describe
from visualization import backtest_charts

pio.renderers.default = 'notebook_connected'
pd.set_option('display.max_columns', None)

### 2. Load data and strategy definition

In [2]:
df = pd.read_parquet(YFINANCE_CLEAN)
backtest_strategy = 'ratio'


### 3. Check describe of the selected strategy

In [3]:


df_strategy = strategy_describe(df, tons_conversion, backtest_strategy=backtest_strategy)

styled_df = df_strategy.style \
    .format(na_rep='—') \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'center')]
    }])
styled_df



,count,mean,std,min,25%,50%,75%,max,coefficient variation
Crude oil/Heating oil,6028.000000,0.782000,0.090600,-1.003700,0.726500,0.780700,0.847200,0.992500,0.115800
Crude oil/Corn,6028.000000,3.070500,0.970900,-2.250700,2.362000,2.782700,3.619300,6.447900,0.316200
Crude oil/Soybean oil,6028.000000,0.595100,0.157000,-0.486200,0.475500,0.580500,0.684300,1.078100,0.263900
Crude oil/Soybean meal,6028.000000,1.442200,0.472500,-0.884500,1.082900,1.359600,1.715900,3.252300,0.327600
Crude oil/Soybean,6028.000000,1.319400,0.376400,-0.917000,1.039900,1.283100,1.542700,2.757000,0.285300
Crude oil/Wheat,6028.000000,2.448900,0.644300,-1.381200,1.989000,2.384200,2.887000,4.605500,0.263100
Heating oil/Crude oil,6028.000000,1.295300,0.161300,-0.996300,1.180200,1.280800,1.376400,3.066600,0.124500
Heating oil/Corn,6028.000000,3.931000,1.135700,1.585900,3.119000,3.808800,4.580700,8.399800,0.288900
Heating oil/Soybean oil,6028.000000,0.764000,0.189600,0.346200,0.626000,0.762100,0.868600,1.365600,0.248100
Heating oil/Soybean meal,6028.000000,1.849400,0.565200,0.606500,1.382200,1.830400,2.212200,3.824300,0.305600


### 4. Input strategy parameters

In [4]:
start_date = '2010-04-01'
end_date = '2025-05-02'

commodity_chosen = 'Soybean'
commodity_ratio = 'Wheat'

down_entry = 2.4
up_exit = 2.5

contract_size = contract_sizes[commodity_chosen]


### 5. Backtest execution

In [5]:

df_trades, position_open = backtest(
    backtest_strategy=backtest_strategy,
    start_date=start_date,
    end_date=end_date,
    df=df,
    up_exit=up_exit,
    down_entry=down_entry,
    commodity_chosen=commodity_chosen,
    commodity_ratio=commodity_ratio,
    tons_conversion=tons_conversion,
    contract_size=contract_size
)


### 6. PnL

In [6]:
df_trades, mtm_trade = pnl_trades(
    df_trades=df_trades,
    df_prices=df,
    commodity_chosen=commodity_chosen,
    tons_conversion=tons_conversion,
    contract_size=contract_size,
    position_open=position_open
)

### 7. Charts

In [7]:
backtest_charts(
    df_prices=df,
    df_trades=df_trades,
    commodity_chosen=commodity_chosen,
    down_entry=down_entry,
    up_exit=up_exit,
    start_date=start_date,
    end_date=end_date,
    mtm_trade=mtm_trade,
    tons_conversion=tons_conversion,
    use_streamlit=False
)


### 8. Performance analysis

In [8]:
metrics = backtest_performance(
    df_trades, df, mtm_trade,
    contract_size=contract_sizes[commodity_chosen],
    tons_conversion=tons_conversion,
    commodity_chosen=commodity_chosen,
    position_open=position_open
)

display(metrics)

,Metric,Value
0,Total Buys,9.00
1,Total Sells,8.00
2,Complete Trades,8.00
3,Open Positions,1.00
4,Realized Profit (USD),15883.96
5,MTM Adjustment (USD),-5222.75
6,Total Profit (USD),10661.21
7,Win Rate (%),75.00
8,Max Drawdown (USD),2973.94
9,Sharpe Ratio,5.35
